In [2]:
import pandas as pd
import numpy as np
import json
from together import Together
import os
from dotenv import load_dotenv
from together import Together

# Load API key from .env file
load_dotenv()
API_KEY = os.getenv("TOGETHER_API_KEY")
client = Together(api_key=API_KEY)

In [3]:
physician_responded_posts_df = pd.read_csv("merged_physician_submissions_100.csv")

In [13]:
physician_responded_posts_df['id']

KeyError: 'id'

In [ ]:
# Prepare storage for results
results = []

# Store output of models separately
results_prompt1_dict = {}
results_prompt2_dict = {}

for _, post in physician_responded_posts_df.iterrows():
    post_id = post.get("clean_link_id", "No id")
    title = post.get("title", "No title")
    link_flair_text = post.get("link_flair_text", "No flair")
    body = post.get("selftext", "No content available")
    num_comments = post.get("num_comments", 0)
    comment_score = post.get("score_comment", 0)

    # Prompt 1: Medical Diagnosis Suggestion
    prompt1 = (
        f"Based on the following symptoms, list possible medical conditions and recommend "
        f"the next diagnostic tests or evaluations a doctor might order. "
        f"Symptoms: {body}. Include brief justifications for each recommendation."
    )

    response1 = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt1}],
    )

    results_prompt1_dict[post_id] = response1.choices[0].message.content

    # Prompt 2: Reddit-Style Physician Response
    prompt2 = (
        f"You're replying to a Reddit post in r/AskDocs. The post asks: {body}. "
        f"Write a comment like a real Reddit user who is a verified physician."
    )

    response2 = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt2}],
    )

    results_prompt2_dict[post_id] = response2.choices[0].message.content

    # Store results in dictionary
    results.append({
        "id": post_id,
        "title": title,
        "link_flair_text": link_flair_text,
        "selftext": body,
        "num_comments": num_comments,
        "comment_score": comment_score,
        "generated_response_prompt1": response1.choices[0].message.content,
        "generated_response_prompt2": response2.choices[0].message.content
    })

# Save model outputs as JSON files
with open("generated_responses_prompt1.json", "w") as f:
    json.dump(results_prompt1_dict, f, indent=4)

with open("generated_responses_prompt2.json", "w") as f:
    json.dump(results_prompt2_dict, f, indent=4)

df_final = pd.DataFrame(results)

df_final.to_csv("final_generated_responses.csv", index=False)

print(f"Processed {len(df_final)} posts. Results saved to 'final_generated_responses.csv'.")
